# 3b. Featurizing the data - BERT based uncased model

## 3b.0 Importing libraries

In [1]:
# Importing libraries

import os
from pathlib import Path
import numpy as np 
import pandas as pd 
import nltk; nltk.download("punkt")
from sklearn.preprocessing import OneHotEncoder
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm   
import warnings
warnings.filterwarnings("ignore")
import math

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3b.1 Data
### 3b.1.1 Gathering data

In [2]:
# Moving to parent directory

os.chdir(Path(os.path.realpath("")).resolve().parents[1])

# Importing the data gathering modules

from src.getter.load_application_and_opportunity import *

fdata = get_interim_data("preprocesseddata")
fdata.head()

,OpportunityId,ApplicationId,ExternalBriefDescription,ExternalDescription,Title,JobCategoryName,IsRejected,IsCandidateInternal,BehaviorCriteria,MotivationCriteria,...,SkillCriteria__trnsfrmrpp,WorkExperiences__trnsfrmrpp,Educations__trnsfrmrpp,LicenseAndCertifications__trnsfrmrpp,Skills__trnsfrmrpp,Motivations__trnsfrmrpp,Behaviors__trnsfrmrpp,StepId__trnsfrmrpp,StepName__trnsfrmrpp,StepGroup__trnsfrmrpp
0,MbzeABKVn06G8irkoHJeIg==,nTzdqGj020CYqTouPocGSg==,"$16.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$16.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,MinimumScaleValue 3 MinimumScaleValueName Inte...,EndMonth None EndYear None JobTitle Call Cente...,Degree Some college Description None Graduatio...,,ScaleValue 4 ScaleValueName Advanced Skill Clo...,Description Inspired to perform well by moneta...,Description Devoted to a task or purpose with ...,K8yQlic+/UiXxBMpOnAoLQ==,Decline,declined
1,7SPt0A57/kyzM9hE9vxDRg==,QVk5MFCZ70WvlZE9FzAW9g==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,MinimumScaleValue 3 MinimumScaleValueName Inte...,EndMonth None EndYear None JobTitle Coordinato...,Degree Diploma Description None GraduationMont...,,ScaleValue 5 ScaleValueName Expert Skill Sales...,,,K8yQlic+/UiXxBMpOnAoLQ==,Decline,declined
2,7SPt0A57/kyzM9hE9vxDRg==,I1kcPlAw3E+rqceh0qrutQ==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,MinimumScaleValue 3 MinimumScaleValueName Inte...,EndMonth None EndYear None JobTitle Direct Car...,Degree HIGH SCHOOL DIPLOMA Description None Gr...,,ScaleValue 4 ScaleValueName Advanced Skill Cas...,,,K8yQlic+/UiXxBMpOnAoLQ==,Decline,declined
3,zolSWBFjWESbfkj8AXLYwA==,VTCXZK6/ZUWJDpxTcm2CRg==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,MinimumScaleValue 3 MinimumScaleValueName Inte...,EndMonth None EndYear 2019.0 JobTitle Package ...,Degree Associate in Early Description None Gra...,,ScaleValue 5 ScaleValueName Expert Skill Cashi...,,,K8yQlic+/UiXxBMpOnAoLQ==,Decline,declined
4,zolSWBFjWESbfkj8AXLYwA==,I6KgcL0jdkG8wBnL1+BZ/g==,"$15.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 Per Hour</strong></p>\n<p><s...,Customer Service Specialist,Customer Service,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,MinimumScaleValue 3 MinimumScaleValueName Inte...,EndMonth None EndYear None JobTitle Warehouse ...,Degree Bachelor of Business Admin Description ...,,ScaleValue 5 ScaleValueName Expert Skill Forkl...,,,K8yQlic+/UiXxBMpOnAoLQ==,Decline,declined


In [3]:
fdata.columns

Index(['OpportunityId', 'ApplicationId', 'ExternalBriefDescription',
       'ExternalDescription', 'Title', 'JobCategoryName', 'IsRejected',
       'IsCandidateInternal', 'BehaviorCriteria', 'MotivationCriteria',
       'EducationCriteria', 'LicenseAndCertificationCriteria', 'SkillCriteria',
       'WorkExperiences', 'Educations', 'LicenseAndCertifications', 'Skills',
       'Motivations', 'Behaviors', 'StepId', 'StepName', 'Tag', 'StepGroup',
       'pass_first_step', 'ExternalBriefDescription__pp',
       'ExternalDescription__pp', 'Title__pp', 'JobCategoryName__pp',
       'BehaviorCriteria__pp', 'MotivationCriteria__pp',
       'EducationCriteria__pp', 'LicenseAndCertificationCriteria__pp',
       'SkillCriteria__pp', 'WorkExperiences__pp', 'Educations__pp',
       'LicenseAndCertifications__pp', 'Skills__pp', 'Motivations__pp',
       'Behaviors__pp', 'StepId__pp', 'StepName__pp', 'StepGroup__pp',
       'ExternalBriefDescription__w2vpp', 'ExternalDescription__w2vpp',
       'Titl

### 3b.1.2 Defining column names for featurization 

In [4]:
# Defining list containing names of the columns

job_column = [
    'ExternalBriefDescription',
    'ExternalDescription', 
    'Title', 
    'JobCategoryName'
]

uid_column = ['OpportunityId', 'ApplicationId']

# Column - 'Tag' Will be added later
can_column = [
    'IsCandidateInternal',
    'BehaviorCriteria', 
    'MotivationCriteria',
    'EducationCriteria', 
    'LicenseAndCertificationCriteria', 
    'SkillCriteria', 
    'WorkExperiences', 
    'Educations', 
    'LicenseAndCertifications', 
    'Skills', 'Motivations', 
    'Behaviors', 
    'StepName', 
    'Tag', 
    'StepGroup',
    'pass_first_step'
]

sel_column = ['IsRejected']

# Defining list of columns based on the type of contents

str_column = [
    'ExternalBriefDescription', 
    'ExternalDescription', 
    'Title', 
    'JobCategoryName', 
    'BehaviorCriteria', 
    'MotivationCriteria', 
    'EducationCriteria', 
    'LicenseAndCertificationCriteria', 
    'SkillCriteria', 
    'WorkExperiences', 
    'Educations', 
    'LicenseAndCertifications', 
    'Skills', 
    'Motivations', 
    'Behaviors', 
    'StepId', 
    'StepName', 
    'StepGroup'
]

bool_column = ['IsCandidateInternal', 'pass_first_step']

float_column = ['Tag']

## 3b.2 Generating embeddings using transformer based large language models
### 3b.2.1 Observations on the data
It was observed in our EDA that there were only 8473 opportunities, which implies the text in the string columns are likely to be repeated for same jobs. Also, even though the the applicaiton ID is unique, it is likely that these same candidate with details may have applied for multiple jobs. In order to reduce the time and space capacities, lets check if the string columns when put together contains any duplicates. 

In [5]:
# Gathering necessary columns of data 

fdata['opportunity__str'] = fdata[[m + "__trnsfrmrpp" 
                                   for m in job_column if m in str_column]
                                   ].agg(" ".join, axis = 1).tolist()

fdata['candidate__str'] = fdata[[m + "__trnsfrmrpp" 
                                 for m in can_column if m in str_column]
                                 ].agg( " ".join, axis = 1).tolist()


job_data = fdata[job_column + ["OpportunityId"]].duplicated()

 # ApplicationID ommitted as it was unique for all values

can_data = fdata['candidate__str'].duplicated()

print(job_data.value_counts(),  can_data.value_counts())
fdata.shape[0]

True     101794
False      8473
Name: count, dtype: int64 candidate__str
False    109754
True        513
Name: count, dtype: int64


110267

With so many (about 90%) duplicate values in candidate opportunities, it is prudent to therefore pass non-duplicates to the BERT and then associate the embeddings back to data.

There wouldn't be significant time improvements(a reduction around 0.6% in the reduction of values) when pass non-duplicates to the BERT in case of the candidate textual information, yet we do it for the sake of uniformity in the code and execution. 

### 3b.2.2 Creating functions that derive bert based embeddings as features

In [7]:
# Code for function:

def modelbased_embedder(data, uid_column_name, str_column, bool_column, float_column, hugging_face_model_name):
    """
    Embeds textual data using a pre-trained transformer model from Hugging Face.

    Parameters:
        data (pd.DataFrame): The input DataFrame containing textual data to be 
        embedded.
        uid_column_name (str): The name of the column in the DataFrame that 
        contains unique identifiers for each row.
        str_column (list): List containing the names(str) of columns onto which
        modelbased_embedder are to be performed.
        bool_column: List containing the names(str) of boolean columns
        float_column: List containing the names(str) of the float data
        hugging_face_model_name (str): The name or path of the pre-trained 
        transformer model from Hugging Face.

    Returns:
        dict_hstack (dict): A dictionary mapping unique identifiers to 
        horizontally stacked BERT embeddings.
        dict_vstack (dict): A dictionary mapping unique identifiers to 
        vertically stacked mean-pooled BERT embeddings.

    Notes:
        1. This function pads the flaot and boolean data to the transformer's 
        vector size (i.e. config.hidden_size) so as to perform vector/tensor
        mean(explained later). Therefore, in the horizontal 
        stacking/concatenating, the vector adds unnecessary dimensions. 
        2. This function uses the specified Hugging Face model to tokenize and 
        embed text data.
        3. The embeddings are computed for each row in the input DataFrame, and 
        the results are stored in dictionaries.
        4. Horizontal stacking (`job_opportunityid_bert_dict_hstack`) concatenates 
        embeddings for each column.
        5. Vertical stacking (`job_opportunityid_bert_dict_vstack`) computes the 
        mean-pooled embeddings across all columns.
        6. The resulting dictionaries can be used for downstream tasks such as 
        machine learning or similarity comparisons.
    """

    # Defining functions that encode and pad boolean and float values

    def encode_and_pad_boolean_columns(data, bool_column, vector_dim = 768):
        """
        Encode bookean columns in a pandas DataFrame using OneHot Encoder
        
        Args:
            data (pandas DataFrame): upon whose boolean columns the encoding is 
            to executed
            bool_column (list): List containing the boolean columns names to be 
            encoded
            vector_dim (int): Dimension of the w2v_vectors
        
        Returns:
            None, modifies the DataFrame in place adding new columns with one 
            hot encoded data
        
        """

        if 'OneHotEncoder' in dir():
            onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
        else:
            from sklearn.preprocessing import OneHotEncoder
            onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
        
        for colname in bool_column:
            data[colname + "__"] = [
                np.pad(x,  
                    (0, vector_dim - (len(x) % vector_dim)), 
                    'constant') for x in onehotencoder.fit_transform(
                        np.reshape(np.array(data[colname]), (-1, 1))
                        )
                    ]

    def pad_float_columns(data, float_column, vector_dim = 768):
        """
        Pads the specified float columns in the fdata pandas DataFrame so that 
        the final value has a length equal to vector_dim

        Args:
            fdata (pandas DataFrame): Data frame containing the float value
            float_column (list): List of column names containing the float data
            vector_dim (int): Dimension of the vector the columns will be 
            padded

        Returns:
            None: Converts/ modifies the data and generates the new columns
        """
        for colname in float_column:
            data[colname + "__"] = [np.pad(
                x, 
                (0, vector_dim - (len(x) % vector_dim)), 
                'constant'
            ) for x in (np.reshape(
                np.array(data[colname]), (-1, 1)
            ))]

    # Main function begins  
    # Applying hugging face model and tokenizer
    # Loading model and tokenizer
    '''Shifting to GPU for faster calculations - as applicable (user should 
    test if frequent transitioning between GPU and CPU, and viceversa 
    increases time-complexity)'''

    # Use the below code on higher config machines
    
    if torch.cuda.is_available():
        device = torch.device("cuda")
    # elif torch.backends.mps.is_available():
    #     device  = torch.device("mps")
    else:
        device = torch.device("cpu") 
    
    if 'AutoTokenizer' in dir() and 'AutoModel' in dir():
        tokenizer = AutoTokenizer.from_pretrained(hugging_face_model_name) 
        model = AutoModel.from_pretrained(hugging_face_model_name).to(device) 

    else:
        from transformers import AutoTokenizer, AutoModel
        tokenizer = AutoTokenizer.from_pretrained(hugging_face_model_name) 
        model = AutoModel.from_pretrained(hugging_face_model_name).to(device)   
    
    # Gathering data 
    data__ = data[
        [uid_column_name]+ 
        [x + "__trnsfrmrpp" for x in str_column] + 
        bool_column + 
        float_column
    ].drop_duplicates() 

    # Applying encode_pad_boolean_columns and pad_float_columns 

    encode_and_pad_boolean_columns(data__, bool_column, vector_dim = model.config.hidden_size)
    pad_float_columns(data__, float_column, vector_dim = model.config.hidden_size)
    
    # Gathering and applying BERT base embedded vector for opportunity columns
    
    dict_hstack = {}
    dict_vstack = {}
    str_data__ = data__[[x + "__trnsfrmrpp" for x in str_column]] # Gathering string data only

    for index, row in tqdm(
        str_data__.iterrows(), desc = "Processing rows", total = len(data__)
    ):
        
        embeddings_values = []
        
        for column in str_data__.columns:
            text = str_data__.at[index, column]

            # Taking care of empty text
            if not text:
                zero_embeddings = np.zeros((model.config.hidden_size,))
                embeddings_values.append(zero_embeddings)
                continue
            
            #Taking care of the blank [] obtained because of the lack of punctuation mark
            if text and text[-1] not in ".?!":
                text+= "."

        
            sentences = nltk.tokenize.sent_tokenize(text)
            
            inputs = tokenizer(
                sentences, 
                padding = True, 
                truncation = True, 
                return_tensors = "pt"
            ).to(device) 
            
            with torch.no_grad():
                output = model(**inputs)
                
            embeddings_values.append(
                np.mean(
                    output.last_hidden_state.mean(dim=1).cpu().numpy(), 
                    axis = 0
                )
            )
        
        # Now adding boolean and float values
        # Adding padded bool_column values to the embeddings_values
        for column in bool_column:
            embeddings_values.append(data__.at[index, column + "__"])
            
        # Adding padded float_column values to the embeddings_values
        for column in float_column:
            embeddings_values.append(data__.at[index, column + "__"])

        # Stacking the embeddings, boolean and float values 
        vector_h = np.hstack(tuple(embeddings_values))
        vector_v = np.mean((tuple(embeddings_values)), axis = 0)
        
        dict_hstack[data__.at[index, uid_column_name]] = vector_h
        dict_vstack[data__.at[index, uid_column_name]] = vector_v

    return dict_hstack, dict_vstack

In [8]:
# Code for function - Batch wise: - Created only for testing

def modelbased_embedder_batched(data, uid_column_name, str_column, bool_column, float_column, hugging_face_model_name, batch_size = 32):
    """
    Embeds textual data using a pre-trained transformer model from Hugging Face.

    Parameters:
        data (pd.DataFrame): The input DataFrame containing textual data to be 
        embedded.
        uid_column_name (str): The name of the column in the DataFrame that 
        contains unique identifiers for each row.
        str_column (list): List containing the names(str) of columns onto which
        modelbased_embedder are to be performed.
        bool_column: List containing the names(str) of boolean columns
        float_column: List containing the names(str) of the float data
        hugging_face_model_name (str): The name or path of the pre-trained 
        transformer model from Hugging Face.

    Returns:
        dict_hstack (dict): A dictionary mapping unique identifiers to 
        horizontally stacked BERT embeddings.
        dict_vstack (dict): A dictionary mapping unique identifiers to 
        vertically stacked mean-pooled BERT embeddings.

    Notes:
        1. This function pads the flaot and boolean data to the transformer's 
        vector size (i.e. config.hidden_size) so as to perform vector/tensor
        mean(explained later). Therefore, in the horizontal 
        stacking/concatenating, the vector adds unnecessary dimensions. 
        2. This function uses the specified Hugging Face model to tokenize and 
        embed text data.
        3. The embeddings are computed for each row in the input DataFrame, and 
        the results are stored in dictionaries.
        4. Horizontal stacking (`job_opportunityid_bert_dict_hstack`) concatenates 
        embeddings for each column.
        5. Vertical stacking (`job_opportunityid_bert_dict_vstack`) computes the 
        mean-pooled embeddings across all columns.
        6. The resulting dictionaries can be used for downstream tasks such as 
        machine learning or similarity comparisons.
    """

    # Defining functions that encode and pad boolean and float values

    def encode_and_pad_boolean_columns(data, bool_column, vector_dim = 768):
        """
        Encode bookean columns in a pandas DataFrame using OneHot Encoder
        
        Args:
            data (pandas DataFrame): upon whose boolean columns the encoding is 
            to executed
            bool_column (list): List containing the boolean columns names to be 
            encoded
            vector_dim (int): Dimension of the w2v_vectors
        
        Returns:
            None, modifies the DataFrame in place adding new columns with one 
            hot encoded data
        
        """

        if 'OneHotEncoder' in dir():
            onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
        else:
            from sklearn.preprocessing import OneHotEncoder
            onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
        
        for colname in bool_column:
            data[colname + "__"] = [
                np.pad(x,  
                    (0, vector_dim - (len(x) % vector_dim)), 
                    'constant') for x in onehotencoder.fit_transform(
                        np.reshape(np.array(data[colname]), (-1, 1))
                        )
                    ]

    def pad_float_columns(data, float_column, vector_dim = 768):
        """
        Pads the specified float columns in the fdata pandas DataFrame so that 
        the final value has a length equal to vector_dim

        Args:
            fdata (pandas DataFrame): Data frame containing the float value
            float_column (list): List of column names containing the float data
            vector_dim (int): Dimension of the vector the columns will be 
            padded

        Returns:
            None: Converts/ modifies the data and generates the new columns
        """
        for colname in float_column:
            data[colname + "__"] = [np.pad(
                x, 
                (0, vector_dim - (len(x) % vector_dim)), 
                'constant'
            ) for x in (np.reshape(
                np.array(data[colname]), (-1, 1)
            ))]

    # Main function begins  
    # Applying hugging face model and tokenizer
    # Loading model and tokenizer
    '''Shifting to GPU for faster calculations - as applicable (user should 
    test if frequent transitioning between GPU and CPU, and viceversa 
    increases time-complexity)'''

    # Use the below code on higher config machines
    
    if torch.cuda.is_available():
        device = torch.device("cuda")
    # elif torch.backends.mps.is_available():
    #     device  = torch.device("mps")
    else:
        device = torch.device("cpu") 
    
    if 'AutoTokenizer' in dir() and 'AutoModel' in dir():
        tokenizer = AutoTokenizer.from_pretrained(hugging_face_model_name) 
        model = AutoModel.from_pretrained(hugging_face_model_name).to(device) 

    else:
        from transformers import AutoTokenizer, AutoModel
        tokenizer = AutoTokenizer.from_pretrained(hugging_face_model_name) 
        model = AutoModel.from_pretrained(hugging_face_model_name).to(device)   
    
    
    # Gathering data 
    data__ = data[
        [uid_column_name]+ 
        [x + "__trnsfrmrpp" for x in str_column] + 
        bool_column + 
        float_column
    ].drop_duplicates()  

    # Applying encode_pad_boolean_columns and pad_float_columns 

    encode_and_pad_boolean_columns(data__, bool_column, vector_dim = model.config.hidden_size)
    pad_float_columns(data__, float_column, vector_dim = model.config.hidden_size)
    
    # Gathering and applying BERT base embedded vector for opportunity columns - batch wise

    # Batch wise - 

    batches = [data__[i:i+batch_size] for i in range(0,len(data__), batch_size)]
    
    dict_hstack = {}
    dict_vstack = {}

    for batched_data in tqdm(batches, desc = "Processing Batches", total= math.ceil(len(data__)/batch_size)):
        
        str_data__ = batched_data[[x + "__trnsfrmrpp" for x in str_column]] # Gathering string data only

        for index, row in str_data__.iterrows():
            
            embeddings_values = []
            
            for column in str_data__.columns:
                text = str_data__.at[index, column]

                # Taking care of empty text
                if not text:
                    zero_embeddings = np.zeros((model.config.hidden_size,))
                    embeddings_values.append(zero_embeddings)
                    continue
                
                #Taking care of the blank [] obtained because of the lack of punctuation mark
                if text and text[-1] not in ".?!":
                    text+= "."

            
                sentences = nltk.tokenize.sent_tokenize(text)
                
                inputs = tokenizer(
                    sentences, 
                    padding = True, 
                    truncation = True, 
                    return_tensors = "pt"
                ).to(device) 
                
                with torch.no_grad():
                    output = model(**inputs)
                    
                embeddings_values.append(
                    np.mean(
                        output.last_hidden_state.mean(dim=1).cpu().numpy(), 
                        axis = 0
                    )
                )
            
            # Now adding boolean and float values
            # Adding padded bool_column values to the embeddings_values
            for column in bool_column:
                embeddings_values.append(batched_data.at[index, column + "__"])
                
            # Adding padded float_column values to the embeddings_values
            for column in float_column:
                embeddings_values.append(batched_data.at[index, column + "__"])

            # Stacking the embeddings, boolean and float values 
            vector_h = np.hstack(tuple(embeddings_values))
            vector_v = np.mean((tuple(embeddings_values)), axis = 0)
            
            dict_hstack[data__.at[index, uid_column_name]] = vector_h
            dict_vstack[data__.at[index, uid_column_name]] = vector_v

    return dict_hstack, dict_vstack

### 3b.2.3 Executing the modelbased_embedder function

In [9]:
# Gathering arguments for the modelbased_embedder function for Opportunity columns
# data = fdata
uid_column_name = 'OpportunityId'
str_col = [x for x in job_column if x in str_column]
bool_col = [x for x in job_column if x in bool_column]
float_col = [x for x in job_column if x in float_column]
hugging_face_model_name = "bert-base-uncased"

# Running the modelbased_embedder function
job_opportunityid_bert_dict_hstack, job_opportunityid_bert_dict_vstack = modelbased_embedder(fdata, uid_column_name, str_col, bool_col, float_col, hugging_face_model_name)

Processing rows: 100%|██████████| 8473/8473 [1:57:07<00:00,  1.21it/s]  


In [10]:
# Gathering arguments for the modelbased_embedder function for Candidate columns
# data = fdata
uid_column_name = 'ApplicationId'
str_col = [x for x in can_column if x in str_column]
bool_col = [x for x in can_column if x in bool_column]
float_col = [x for x in can_column if x in float_column]
hugging_face_model_name = "bert-base-uncased"

# Running the modelbased_embedder function
can_applicationid_bert_dict_hstack, can_applicationid_bert_dict_vstack = modelbased_embedder(fdata, uid_column_name, str_col, bool_col, float_col, hugging_face_model_name)

Processing rows: 100%|██████████| 110267/110267 [20:42:38<00:00,  1.48it/s]  


In [11]:
# Printing a random rows to see the shape of the dictionararies generated

print(
    can_applicationid_bert_dict_hstack['nTzdqGj020CYqTouPocGSg=='].shape, 
    job_opportunityid_bert_dict_hstack['MbzeABKVn06G8irkoHJeIg=='].shape
)

print(
    can_applicationid_bert_dict_vstack['nTzdqGj020CYqTouPocGSg=='].shape, 
    job_opportunityid_bert_dict_vstack['MbzeABKVn06G8irkoHJeIg=='].shape
)

(12288,) (3072,)
(768,) (768,)


## 3b.3 Saving data for futher analysis

In [12]:
# Importing modules that save the data 
from src.getter.save_application_and_opportunity import save_interim_data

# Adding dictionaries into the variables for pickle
# Creating dictionary
bert_dict = {}

# Adding dictionaries
bert_dict[
    'job_opportunityid_bert_dict_hstack'
    ] = job_opportunityid_bert_dict_hstack
bert_dict[
    'can_applicationid_bert_dict_hstack'
    ] = can_applicationid_bert_dict_hstack

bert_dict[
    'job_opportunityid_bert_dict_vstack'
    ] = job_opportunityid_bert_dict_vstack
bert_dict[
    'can_applicationid_bert_dict_vstack'
    ] = can_applicationid_bert_dict_vstack

# Saving variables dictionary

save_interim_data(bert_dict, "bert_data_dictionary")